In [2]:
from astropy import coordinates as coord
from astropy import units as u
from astropy.coordinates import Angle
from gala.coordinates import MagellanicStreamNidever08
import numpy as np
import sys
import pandas as pd
import pyregion


## Filter all sky tiles to just the GASKAP target set

This notebook will take a set of all sky tiles and filter them down to a defined range. The range file is a csv with each row specifying a rule as follows:

* min_l - The minimum Magellanic longitude of the rule (inclusive)
* num_l - The number of 5 deg longitude bands that the rule covers
* max_l - The maximum Magellanic longitude of the rule (inclusive)
* min_b - The minimum Magellanic latitude (inclusive) to be included in this longitude range
* max_b - The minimum Magellanic latitude (inclusive) to be included in this longitude range
* num_b - The number of 5 deg tiles to be included

In [8]:
def find_ms_b_range(tile_ranges, ms_l):
    for row in tile_ranges:
        if row[0] <= ms_l and row[2] >= ms_l:
            #print (row[0], row[2], row[3], row[4])
            return row[3], row[4]
    return None
        
#find_ms_b_range(tile_ranges.to_numpy(), -15)

In [9]:
def filter_regions(np_tile_ranges, regfile):
    clippedarray=np.zeros(11)
    #convert from gal to mag and check elements
    for i in range(0,len(regfile),2):
        #load in coordinates as j2000, references one of the tile vertices i think,should change to centre where the text is placed
        c = coord.FK5(ra=float(regfile[i+1,1])*u.deg, dec=float(regfile[i+1,2])*u.deg)
        #convert to mag coords
        ms = c.transform_to(MagellanicStreamNidever08)
        # Find the appropriate range
        ms_b_range = find_ms_b_range(np_tile_ranges, ms.L.value)
        if ms_b_range and ms_b_range[0] <= ms.B.value <= ms_b_range[1]:
            print ("tile is inside at L={:.2f} B={:.2f}".format(ms.L.value, ms.B.value))
            clippedarray=np.vstack((clippedarray,regfile[i]))
            clippedarray=np.vstack((clippedarray,regfile[i+1]))
            clippedarray[-1,-5:] = ""

    # Remove the first, template row
    clippedarray=clippedarray[1:,]
    return clippedarray


## Produce region files for the pilot survey

In [10]:
regfileloc = 'gaskap_mag_2021.reg'
#regfileloc = 'croppedtilefield.reg'
importreg= pd.read_csv(regfileloc, delim_whitespace=True, header=None)
regfile=importreg.to_numpy(dtype='str')


In [11]:
regfile

array([['j2000;polygon', '272.31564199999997', '-1.27862', ...,
        '-0.968601', '272.31564199999997', '-1.27862'],
       ['j2000;text', '468.3623', '1.211185', ..., 'nan', 'nan', 'nan'],
       ['j2000;polygon', '266.906294', '-1.451144', ..., '-1.279169',
        '266.906294', '-1.451144'],
       ...,
       ['j2000;text', '421.45900199999994', '31.682238', ..., 'nan',
        'nan', 'nan'],
       ['j2000;polygon', '93.692874', '29.209044', ...,
        '28.854540999999998', '93.692874', '29.209044'],
       ['j2000;text', '423.03196299999996', '31.401690999999996', ...,
        'nan', 'nan', 'nan']], dtype='<U21')

In [12]:
tile_ranges = pd.read_csv('pilot_tile_range.csv')
tile_ranges

,min_l,num_l,max_l,min_b,max_b,num_b
0,-15,1,-10,-10,-3,2.4
1,-10,2,-5,-8,-5,1.6


In [13]:
np_tile_ranges = tile_ranges.to_numpy()
clippedarray = filter_regions(np_tile_ranges, regfile)

tile is inside at L=-10.74 B=-3.02
tile is inside at L=-6.80 B=-7.36
tile is inside at L=-10.30 B=-3.41
tile is inside at L=-5.80 B=-7.98
tile is inside at L=-12.25 B=-9.48
tile is inside at L=-8.22 B=-6.00
tile is inside at L=-11.04 B=-3.08
tile is inside at L=-11.91 B=-7.21
tile is inside at L=-10.10 B=-6.77
tile is inside at L=-10.00 B=-5.65
tile is inside at L=-11.35 B=-3.74
tile is inside at L=-14.16 B=-4.21
tile is inside at L=-13.90 B=-3.10


In [14]:

np.savetxt('pilottilefield_202110.reg',clippedarray,fmt='%s')
np.savetxt('pilottilefield_202110_notext.reg',clippedarray[0::2],fmt='%s')

## Produce region files for the entire survey

In [15]:
regfileloc = 'gaskap_mag_2021.reg'
#regfileloc = 'croppedtilefield.reg'
importreg= pd.read_csv(regfileloc, delim_whitespace=True, header=None)
regfile=importreg.to_numpy(dtype='str')

tile_ranges = pd.read_csv('tile_range.csv')
tile_ranges

,min_l,num_l,max_l,min_b,max_b,num_b,Unnamed: 6,Unnamed: 7
0,-130,1,-125,-10,5,4,-12.5,7.5
1,-125,2,-115,-10,20,7,-12.5,22.5
2,-115,3,-100,-10,15,6,-12.5,17.5
3,-100,3,-85,-10,10,5,-12.5,12.5
4,-85,4,-65,-5,5,3,-7.5,7.5
5,-65,3,-50,-5,10,4,-7.5,12.5
6,-50,1,-45,-10,10,5,-12.5,12.5
7,-45,3,-30,-25,10,8,-27.5,12.5
8,-30,7,5,-20,10,7,-22.5,12.5
9,5,4,25,-25,10,8,-27.5,12.5


In [16]:
np_tile_ranges = tile_ranges.to_numpy()
clippedarray = filter_regions(np_tile_ranges, regfile)

np.savetxt('tilefield_202110.reg',clippedarray,fmt='%s')
np.savetxt('tilefield_202110_notext.reg',clippedarray[0::2],fmt='%s')

tile is inside at L=10.50 B=8.60
tile is inside at L=8.04 B=7.92
tile is inside at L=6.57 B=8.20
tile is inside at L=6.13 B=9.55
tile is inside at L=17.83 B=9.62
tile is inside at L=14.02 B=7.19
tile is inside at L=10.39 B=5.27
tile is inside at L=7.23 B=4.00
tile is inside at L=5.06 B=3.64
tile is inside at L=4.21 B=4.46
tile is inside at L=4.42 B=6.52
tile is inside at L=5.18 B=9.59
tile is inside at L=18.84 B=7.55
tile is inside at L=14.91 B=4.75
tile is inside at L=10.96 B=2.36
tile is inside at L=7.10 B=0.53
tile is inside at L=3.90 B=-0.58
tile is inside at L=2.30 B=-0.50
tile is inside at L=2.39 B=1.14
tile is inside at L=3.28 B=4.09
tile is inside at L=4.36 B=7.88
tile is inside at L=22.51 B=7.86
tile is inside at L=18.56 B=4.53
tile is inside at L=14.55 B=1.45
tile is inside at L=10.37 B=-1.22
tile is inside at L=5.94 B=-3.35
tile is inside at L=1.75 B=-4.92
tile is inside at L=0.00 B=-5.08
tile is inside at L=0.85 B=-2.94
tile is inside at L=2.24 B=0.62
tile is inside at L=3.